**TODO :**
- tf-idf if not oom ?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src

/home/theo/kaggle/foursquare/src


In [3]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# **Librairies**

In [4]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch

torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 2080 Ti'

<IPython.core.display.Javascript object>

In [5]:
import gc
import random
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from params import DEBUG, OUT_PATH, IS_TEST
from ressources import *
from matching import *

random.seed(13)
warnings.simplefilter("ignore")
pd.options.display.max_columns = 500

<IPython.core.display.Javascript object>

## Load Data

In [6]:
if IS_TEST:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_test.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_test.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_test.csv")
else:
    train = load_cleaned_data(OUT_PATH + "cleaned_data_train.csv")
    p1 = pd.read_csv(OUT_PATH + "p1_yv_train.csv")
    p2 = pd.read_csv(OUT_PATH + "p2_yv_train.csv")

<IPython.core.display.Javascript object>

In [7]:
DEBUG = False

<IPython.core.display.Javascript object>

In [8]:
if DEBUG:
    p1 = p1.head(1000000).copy()
    p2 = p2.head(1000000).copy()

<IPython.core.display.Javascript object>

### Prepare

In [9]:
train["idx"] = np.arange(len(train))

<IPython.core.display.Javascript object>

In [10]:
# add other columns - needed for FE
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "categories",
    "category_simpl",
    "categories_split",
    "cat2",
    "idx",
    "state",
    "zip",
]
p1 = p1[["id"]].merge(train[cols], on="id", how="left")
p2 = p2[["id"]].merge(train[cols], on="id", how="left")

<IPython.core.display.Javascript object>

In [11]:
# check for flipped sign on longitude - this may help test data a lot; test it? Move this code up to apply to "train"
dist = distance(
    np.array(p1["latitude"]),
    np.array(p1["longitude"]),
    np.array(p2["latitude"]),
    np.array(p2["longitude"]),
)
df = pd.DataFrame(dist)
df.columns = ["dist"]
df["dist"] = df["dist"].astype("int32")
df["dist1"] = (111173.444444444 * np.abs(p1["latitude"] - p2["latitude"])).astype(
    "int32"
)
df["dist2"] = np.sqrt(np.maximum(0, (1.0 * df["dist"]) ** 2 - df["dist1"] ** 2)).astype(
    "int32"
)
idx = (
    (df["dist1"] < 10000)
    & (df["dist2"] > 1000000)
    & (np.abs(p1["longitude"] + p2["longitude"]) < 0.1)
) & (p1["country"] == p2["country"])
# this selects only 3 cases in train data, but possibly more in test, so keep it becasue it is basically free
print("flipped sign of longitude for", idx.sum(), "points")
p1["longitude"].loc[idx] *= -1  # flip(correct) sign
del df, idx, dist
gc.collect()

flipped sign of longitude for 1 points


3224

<IPython.core.display.Javascript object>

### Youri & Vincent

In [12]:
from fe import FE1

<IPython.core.display.Javascript object>

In [13]:
%%time
df = FE1(p1, p2) # call FE1

- Distances
- Processing column : name
- Processing column : categories
- Processing column : address
- Nan features
- Matching
- Category match
- Ratios
- Count encodings
CPU times: user 12min 52s, sys: 13 s, total: 13min 5s
Wall time: 13min 4s


<IPython.core.display.Javascript object>

In [14]:
df.insert(0, "id_1", p1["id"].values)
df.insert(1, "id_2", p2["id"].values)

<IPython.core.display.Javascript object>

In [15]:
df.head()

,id_1,id_2,dist,dist1,dist2,country,cat2a,cat2b,name_pi1,name_lcs2,name_lcs,name_pi1_r1,name_lcs2_r1,name_lcs2_r2,name_lcs_r1,name_lcs_r2,name_r3,name_lcs_r4,categories_pi1,categories_lcs2,categories_lcs,categories_pi1_r1,categories_lcs2_r1,categories_lcs2_r2,categories_lcs_r1,categories_lcs_r2,categories_r3,categories_lcs_r4,address_pi1,address_lcs2,address_lcs,address_pi1_r1,address_lcs2_r1,address_lcs2_r2,address_lcs_r1,address_lcs_r2,address_r3,address_lcs_r4,city_NA,address_NA,phone_m10,url_m5,same_cat_simpl,dist_r1,dist_r2,id_cc_min,id_cc_max,name_cc_min,name_cc_max
0,E_000001272c6c5d,E_00f3e03e4a63da,36320,4910,36320,11,1,1,4,4,8,0.31,0.31,0.22,0.62,0.44,0.72,0.50,4,4,4,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1,2,3,0.09,0.18,0.12,0.27,0.18,0.65,0.67,0,0,0,0,1,600.845032,800.000000,39,48,48,73
1,E_000001272c6c5d,E_051d16ff51c824,32860,2700,32860,11,1,1,0,1,2,0.00,0.10,0.06,0.20,0.11,0.56,0.50,0,1,3,0.0,0.25,0.04,0.75,0.12,0.16,0.33,11,11,11,1.00,1.00,0.50,1.00,0.50,0.50,1.00,0,0,0,0,0,543.571899,243.691925,36,48,48,66
2,E_0e7fa023e1b41f,E_000001272c6c5d,18030,4020,18030,11,1,1,4,4,5,0.44,0.44,0.22,0.56,0.28,0.50,0.80,4,4,4,1.0,1.00,1.00,1.00,1.00,1.00,1.00,0,1,3,0.00,0.09,0.06,0.27,0.18,0.65,0.33,0,0,0,0,1,297.867401,444.857758,39,48,48,80
3,E_000001272c6c5d,E_143cae76651459,32860,1100,32860,11,1,1,4,4,7,0.33,0.33,0.22,0.58,0.39,0.67,0.57,0,1,3,0.0,0.25,0.06,0.75,0.18,0.24,0.33,0,0,0,0.00,0.00,0.00,0.00,0.00,0.09,0.00,1,1,0,0,0,543.571899,243.691925,36,48,36,48
4,E_000001272c6c5d,E_1820b3153d5ef2,9900,8960,4450,11,1,1,4,4,7,0.44,0.44,0.22,0.78,0.39,0.50,0.57,0,1,3,0.0,0.25,0.10,0.75,0.30,0.40,0.33,1,8,8,0.09,0.73,0.44,0.73,0.44,0.61,1.00,0,0,0,0,0,163.021912,72.699791,48,59,48,1807


<IPython.core.display.Javascript object>

### Théo

In [16]:
from data.features import (
    is_equal,
    is_included,
    levenshtein,
    wratio,
    #     haversine_distance,
    #     manhattan_distance,
    #     angular_distance,
    #     euclidian_distance,
    #     angular_distance_l2,
    compute_string_distance,
    tf_idf_similarity,
)

<IPython.core.display.Javascript object>

In [17]:
def compute_string_distances(df, string_columns, verbose=0):
    functions = [
#         gesh,
        levenshtein,
#         wratio,
    ]

    features = []
    for col in string_columns:
        df[col + "_1"].fillna("", inplace=True)
        df[col + "_2"].fillna("", inplace=True)

        df[f"{col}_len_1"] = df[col + "_1"].apply(len)
        df[f"{col}_len_2"] = df[col + "_2"].apply(len)

        df[f"{col}_len_diff"] = np.abs(df[f"{col}_len_1"] - df[f"{col}_len_2"])
        features.append(f"{col}_len_diff")

        for fct in functions:
            name = fct.__name__
            if verbose:
                print(f"- Column : {col}  -  Function : {name}")

            df[col + "_" + name] = df[[col + "_1", col + "_2"]].apply(
                lambda x: compute_string_distance(fct, x[0], x[1]), axis=1
            )
            features.append(col + "_" + name)

        # Normalize
        df[f"{col}_levenshtein"] = df[f"{col}_levenshtein"] / df[
            [f"{col}_len_1", f"{col}_len_2"]
        ].max(axis=1)

    return features


<IPython.core.display.Javascript object>

In [18]:
def compute_position_distances(df, functions):
    features = []

    lats_1 = df["latitude_1"]
    longs_1 = df["longitude_1"]
    lats_2 = df["latitude_2"]
    longs_2 = df["longitude_2"]

    for fct in functions:
        name = fct.__name__
        df[name] = fct(lats_1, longs_1, lats_2, longs_2)
        df[name + "_s"] = fct(longs_1, lats_1, lats_2, longs_2)
        df[name + "_min"] = df[[name, name + "_s"]].min(1)

        df.drop([name, name + "_s"], axis=1, inplace=True)

        features += [name + "_min"]

    return features

<IPython.core.display.Javascript object>

In [19]:
def angular_distance(lat1, long1, lat2, long2):
    delta_long = pd.concat([np.abs(long2 - long1), 360 - np.abs(long2 - long1)], 1).min(
        1
    )
    return np.abs(lat2 - lat1) + delta_long


def angular_distance_l2(lat1, long1, lat2, long2):
    delta_long = pd.concat([np.abs(long2 - long1), 360 - np.abs(long2 - long1)], 1).min(
        1
    )
    return np.sqrt(np.clip((lat2 - lat1) ** 2 + delta_long**2, 0, 100000))

<IPython.core.display.Javascript object>

In [25]:
def feature_engineering_theo(df, df_p):
    features = []

    for col in ["name", "address", "url"]:
        df.loc[df[col] == "", col] = np.nan

    #     print('- Computing nan features')
    #     features += compute_nan_features(df_p, NAN_COLS)

    #     for name, folder in NN_FT_FOLDERS:
    #         print(f'- Adding features using model {name}')
    #         nn_preds = np.load(OUT_PATH + f"fts_{name}.npy").astype(np.float16)
    #         nn_preds = torch.from_numpy(nn_preds).cuda()

    #         features += compute_nn_distances(df_p, nn_preds, suffix="_" + name)

    #         del nn_preds
    #         gc.collect()
    #         torch.cuda.empty_cache()

    print("- Computing position distances")
    DIST_FCTS = [angular_distance, angular_distance_l2]
    features += compute_position_distances(df_p, DIST_FCTS)

    #     TF_IDF_COLS = ["name", "address", "url"]
    #     TF_IDF_PARAMS = [
    #         ((3, 3), "char_wb"),  # char trigrams
    #     ]
    #     for col in TF_IDF_COLS:
    #         for ngram_range, analyzer in TF_IDF_PARAMS:
    #             ft_name = f"{col}_tf_idf_{ngram_range[0]}{ngram_range[1]}_{analyzer}_sim"
    #             print(f"- Computing feature {ft_name}")

    #             tf_idf = TfidfVectorizer(
    #                 use_idf=False, ngram_range=ngram_range, analyzer=analyzer
    #             )
    #             tf_idf_mat = tf_idf.fit_transform(cudf.from_pandas(df[col].fillna("nan")))

    #             df_p[ft_name] = tf_idf_similarity(df_p, tf_idf_mat)
    #             features.append(ft_name)

    if not isinstance(df_p, pd.DataFrame):
        df_p = df_p.to_pandas()

    FEATURES_SAME = [
        ("state", is_equal),
        ("zip", is_included),
        ("city", is_included),
    ]

    for col, fct in FEATURES_SAME:
        print(f"- Computing feature same_{col}")
        df_p[f"same_{col}"] = (
            df_p[[f"{col}_1", f"{col}_2"]]
            .fillna("")
            .apply(lambda x: fct(x[0], x[1]), axis=1)
            .astype(float)  # parallel_apply
        )

        features.append(f"same_{col}")

    STRING_DIST_COLS = ["name", "address", "url"]
    features += compute_string_distances(df_p, STRING_DIST_COLS, verbose=1)

    to_keep = ["id_1", "id_2"] + features
    df_p.drop([c for c in df_p.columns if c not in to_keep], axis=1, inplace=True)

    return df_p, features

<IPython.core.display.Javascript object>

In [26]:
import cudf
from cuml.feature_extraction.text import TfidfVectorizer

<IPython.core.display.Javascript object>

In [27]:
cols = [
    "id",
    "name",
    "latitude",
    "longitude",
    "address",
    "country",
    "url",
    "phone",
    "city",
    "state",
    "zip",
    "categories",
    "idx",
]
pairs = pd.concat([p1[cols], p2[cols]], axis=1)
pairs.columns = [c + "_1" for c in cols] + [c + "_2" for c in cols]

<IPython.core.display.Javascript object>

In [28]:
%%time

df_theo, fts_theo = feature_engineering_theo(train.copy(), pairs)

- Computing position distances
- Computing feature same_state
- Computing feature same_zip
- Computing feature same_city
- Column : name  -  Function : levenshtein
- Column : address  -  Function : levenshtein
- Column : url  -  Function : levenshtein
CPU times: user 12min 35s, sys: 23.1 s, total: 12min 58s
Wall time: 12min 58s


<IPython.core.display.Javascript object>

## Merge & Save

In [29]:
df_merged = df.merge(df_theo, on=["id_1", "id_2"])

<IPython.core.display.Javascript object>

In [30]:
df_merged.head()

,id_1,id_2,dist,dist1,dist2,country,cat2a,cat2b,name_pi1,name_lcs2,name_lcs,name_pi1_r1,name_lcs2_r1,name_lcs2_r2,name_lcs_r1,name_lcs_r2,name_r3,name_lcs_r4,categories_pi1,categories_lcs2,categories_lcs,categories_pi1_r1,categories_lcs2_r1,categories_lcs2_r2,categories_lcs_r1,categories_lcs_r2,categories_r3,categories_lcs_r4,address_pi1,address_lcs2,address_lcs,address_pi1_r1,address_lcs2_r1,address_lcs2_r2,address_lcs_r1,address_lcs_r2,address_r3,address_lcs_r4,city_NA,address_NA,phone_m10,url_m5,same_cat_simpl,dist_r1,dist_r2,id_cc_min,id_cc_max,name_cc_min,name_cc_max,angular_distance_min,angular_distance_l2_min,same_state,same_zip,same_city,name_len_diff,name_levenshtein,address_len_diff,address_levenshtein,url_len_diff,url_levenshtein
0,E_000001272c6c5d,E_00f3e03e4a63da,36320,4910,36320,11,1,1,4,4,8,0.31,0.31,0.22,0.62,0.44,0.72,0.50,4,4,4,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1,2,3,0.09,0.18,0.12,0.27,0.18,0.65,0.67,0,0,0,0,1,600.845032,800.000000,39,48,48,73,0.935697,0.865765,NaN,NaN,0.0,5,0.611111,6,0.882353,0,NaN
1,E_000001272c6c5d,E_051d16ff51c824,32860,2700,32860,11,1,1,0,1,2,0.00,0.10,0.06,0.20,0.11,0.56,0.50,0,1,3,0.0,0.25,0.04,0.75,0.12,0.16,0.33,11,11,11,1.00,1.00,0.50,1.00,0.50,0.50,1.00,0,0,0,0,0,543.571899,243.691925,36,48,48,66,0.776170,0.736614,NaN,0.0,0.0,8,0.888889,11,0.500000,0,NaN
2,E_0e7fa023e1b41f,E_000001272c6c5d,18030,4020,18030,11,1,1,4,4,5,0.44,0.44,0.22,0.56,0.28,0.50,0.80,4,4,4,1.0,1.00,1.00,1.00,1.00,1.00,1.00,0,1,3,0.00,0.09,0.06,0.27,0.18,0.65,0.33,0,0,0,0,1,297.867401,444.857758,39,48,48,80,0.483910,0.427897,0.0,0.0,0.0,9,0.722222,6,0.823529,0,NaN
3,E_000001272c6c5d,E_143cae76651459,32860,1100,32860,11,1,1,4,4,7,0.33,0.33,0.22,0.58,0.39,0.67,0.57,0,1,3,0.0,0.25,0.06,0.75,0.18,0.24,0.33,0,0,0,0.00,0.00,0.00,0.00,0.00,0.09,0.00,1,1,0,0,0,543.571899,243.691925,36,48,36,48,0.767017,0.751513,NaN,NaN,NaN,6,0.611111,11,NaN,0,NaN
4,E_000001272c6c5d,E_1820b3153d5ef2,9900,8960,4450,11,1,1,4,4,7,0.44,0.44,0.22,0.78,0.39,0.50,0.57,0,1,3,0.0,0.25,0.10,0.75,0.30,0.40,0.33,1,8,8,0.09,0.73,0.44,0.73,0.44,0.61,1.00,0,0,0,0,0,163.021912,72.699791,48,59,48,1807,0.232497,0.165039,0.0,0.0,0.0,9,0.611111,7,0.555556,13,NaN


<IPython.core.display.Javascript object>

In [31]:
# import matplotlib.pyplot as plt

# plt.scatter(df_merged["angular_distance_min"], df_merged["dist_r1"])

<IPython.core.display.Javascript object>

In [33]:
corrs = df_merged.corr()
for col in corrs.columns:
    print(col, corrs.loc[corrs[col] > 0.9, col])
    print("")

dist dist       1.000000
dist2      0.995595
dist_r1    0.955195
dist_r2    0.979753
Name: dist, dtype: float64

dist1 dist1    1.0
Name: dist1, dtype: float64

dist2 dist       0.995595
dist2      1.000000
dist_r1    0.947035
dist_r2    0.973689
Name: dist2, dtype: float64

name_pi1 name_pi1    1.0
Name: name_pi1, dtype: float64

name_lcs2 name_lcs2    1.0
Name: name_lcs2, dtype: float64

name_lcs name_lcs    1.0
Name: name_lcs, dtype: float64

name_pi1_r1 name_pi1_r1    1.0
Name: name_pi1_r1, dtype: float64

name_lcs2_r1 name_lcs2_r1    1.0
Name: name_lcs2_r1, dtype: float64

name_lcs2_r2 name_lcs2_r2    1.000000
name_lcs_r2     0.930759
Name: name_lcs2_r2, dtype: float64

name_lcs_r1 name_lcs_r1    1.0
Name: name_lcs_r1, dtype: float64

name_lcs_r2 name_lcs2_r2    0.930759
name_lcs_r2     1.000000
Name: name_lcs_r2, dtype: float64

name_r3 name_r3    1.0
Name: name_r3, dtype: float64

name_lcs_r4 name_lcs_r4    1.0
Name: name_lcs_r4, dtype: float64

categories_pi1 categories_pi1    

<IPython.core.display.Javascript object>

In [34]:
if not DEBUG:
    if IS_TEST:
        df_merged.to_csv(OUT_PATH + "features_test_1.csv", index=False)
    else:
        df_merged.to_csv(OUT_PATH + "features_train_1.csv", index=False)

<IPython.core.display.Javascript object>